# Splątane sieci neuronowe CNN – architektura Xception 
---
autor: mgr inż. Grzegorz Kossakowski

## 1. Opis architektury

Xception [1][2][3] jest architekturą splątanych sieci neuronowych CNN. Pełna nazwa architektury to Extreme Inception. Powstał w 2017 roku jako ewolucja architektury Inception i został stworzony przez firmę Google. Architektura Xception okazała się bardziej wydajna od VGG-16, ResNet i Inception v3. Głęboko rozdzielny sploty są uznawane za znacznie bardziej wydajne pod względem czasu obliczeń.
Znakiem firmowym Xception jest wykorzystanie głęboko rozdzielnych splotów. Jest to bardzo wydajna architektura oparta na dwóch krokach:
- Głęboko rozdzielny splot
- Splot punktowy


Ogólna architektura składa się z trzech przepływów:
- Przepływ wejściowy
- Przepływ środkową, w której proces jest powtarzany osiem razy
- Przepływ wyjściowy. 



## 2. Pobranie potrzebnych bibliotek
Kolejnym krokiem jest wczytanie wszystkich potrzebnych bibliotek, dzięki którym będzie możliwe wykorzystanie ich w procesie klasyfikacji.

In [2]:
TF_ENABLE_ONEDNN_OPTS=0
from astropy.io import fits
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam
from keras import Sequential
from tests.test_layers import Dense, Flatten
from keras.applications import Xception
import pandas as pd

## 3. Pobranie danych z pliku fits
Dlatego że wcześniej podzieliliśmy dane na odpowiednie części, teraz pobieramy dwa zbiory. Pierwszy z nich to zbiór, na którym będziemy uczyć nasz model. Drugi to zbiór walidacyjny.

In [3]:
hdu_train = fits.open('Data/train.fits')
hdu_valid = fits.open('Data/valid.fits')
x_train = hdu_train[0].data
y_train = hdu_train[1].data
x_valid = hdu_valid[0].data
y_valid = hdu_valid[1].data

In [4]:
x_train.shape, x_valid.shape, type(x_train)

((11350, 256, 256, 3), (2838, 256, 256, 3), numpy.ndarray)

## 4. Ustawienie sposobu nauki
Modele, które używany są już wstępnie wyuczone, dlatego chciałem sprawdzić, jak dany model będzie się zachowywał w dwóch przypadkach. Pierwszy przypadek gdy wartość fullStudy zostanie ustawiona na false wtedy model będzie wykorzystywał wcześniej nauczony model i na ostatnich warstwach będzie douczał tylko danymi astronomicznymi. Gdy ustawimy wartość na true, model od początku będzie, wykonał naukę architektury. Wcześniejsza nauka nie będzie brana pod uwagę. Pozwoli to ocenić, który sposób jest bardziej efektywny.

In [5]:
fullStudy = False

## 5. Pobranie danych 
W tym kroku pobieramy dane, a następnie przygotowujemy je do klasyfikacji. Modele głębokiej sieci neuronowej [4] wymaga danych z zakresu 0..1, dlatego wszystkie wartości w danych są dzielone przez 255. Powodem takiego zachowania jest fakt, że dane obrazów są przechowywane w zakresie liczb 0..255. Dzielenie przez 255 powoduje, że dane zostaną zapisane w zakresie od 0..1, zgodnie z wymaganiami modelu.

In [6]:
reduceLR = ReduceLROnPlateau(monitor='accuracy', factor=.001, patience=1, min_delta=0.01, mode="auto")
x_train = x_train / 255.0
x_valid = x_valid / 255.0

## 6. Budowa modelu.
Model został stworzony w 2017 roku. Jest to model, który powstał na podstawie modelu Inception. Po wykonaniu warstw splątanych następuje przejście przez warstwę flatten. Zadaniem tej warstwy jest spłaszczenie obrazu z wymiarów, które zostały po przejściu przez warstwy splątane na pojedynczy ciąg. Ostatnią warstwą jest gęsto połączona warstwa wyjściowa. W naszym modelu klasyfikacja odbywa się dla 10 kategorii dlatego właśnie taka.

In [7]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(256, 256,3))
base_model.trainable = fullStudy
model_optimizer = Adam(learning_rate=0.001)

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(10, activation="softmax"))

model.compile(optimizer=model_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 10)                1310730   
                                                                 
Total params: 22172210 (84.58 MB)
Trainable params: 1310730 (5.00 MB)
Non-trainable params: 20861480 (79.58 MB)
_________________________________________________________________


## 7. Uczenie
W tym momencie model zaczyna proces uczenia. Czyli otrzymuje dwa zbiory danych i etykiet. Pierwszy z nich to dane, na podstawie których model się uczy. Drugi mniejszy zbiór jest zbiorem walidacyjnym, który pozwala na sprawdzenie postępów w nauce, na danych, których model jeszcze nie widział. Pozwala to ocenić postępy w nauce już w czasie uczenia. Kolejny zbiór danych zostanie wykorzystany na końcu celem ostatecznego sprawdzenia poprawności działania modelu.

In [8]:
history = model.fit(x_train, y_train, epochs=10, callbacks=[reduceLR],validation_data=(x_valid, y_valid))

Epoch 1/10
355/355 [==============================] - 450s 1s/step - loss: 3.6741 - accuracy: 0.4576 - val_loss: 2.9911 - val_accuracy: 0.4961 - lr: 0.0010
Epoch 2/10
355/355 [==============================] - 418s 1s/step - loss: 2.2864 - accuracy: 0.5997 - val_loss: 2.7150 - val_accuracy: 0.5536 - lr: 0.0010
Epoch 3/10
355/355 [==============================] - 416s 1s/step - loss: 1.5939 - accuracy: 0.6880 - val_loss: 4.1401 - val_accuracy: 0.4824 - lr: 0.0010
Epoch 4/10
355/355 [==============================] - 415s 1s/step - loss: 1.3554 - accuracy: 0.7256 - val_loss: 3.2788 - val_accuracy: 0.5395 - lr: 0.0010
Epoch 5/10
355/355 [==============================] - 414s 1s/step - loss: 1.0843 - accuracy: 0.7733 - val_loss: 3.2492 - val_accuracy: 0.5937 - lr: 0.0010
Epoch 6/10
355/355 [==============================] - 414s 1s/step - loss: 0.9025 - accuracy: 0.8012 - val_loss: 3.5235 - val_accuracy: 0.5550 - lr: 0.0010
Epoch 7/10
355/355 [==============================] - 414s 1s/st

## 8. Zapis architektury
Jednak my nie będziemy testować od razu naszego modelu. Do tego celu przygotujemy oddzielny notebook. Dlatego, aby nie utracić naszej pracy, zapiszemy nas wyuczony model do pliku.

In [9]:
if fullStudy == True:
    model.save('Models/Xception_full.keras')
else:
    model.save('Models/Xception.keras')

## 9. Zapis otrzymanych danych podczas nauki
Po zakończeniu uczenia zapisujemy dane, które otrzymaliśmy podczas uczenie do pliku CSV. Pozwoli nam to później przeanalizować dane w późniejszym czasie.

In [10]:
historyModelLearning = pd.DataFrame()
historyModelLearning['loss'] = history.history['loss']
historyModelLearning['accuracy'] = history.history['accuracy']
historyModelLearning['val_loss'] = history.history['val_loss']
historyModelLearning['val_accuracy'] = history.history['val_accuracy']
if fullStudy == True:
    historyModelLearning.to_csv('ResultLearning/Xception_full.csv', index=True)
else:
    historyModelLearning.to_csv('ResultLearning/Xception.csv', index=True)

## Literatura
1. https://maelfabien.github.io/deeplearning/xception/#
2. https://towardsdatascience.com/review-xception-with-depthwise-separable-convolution-better-than-inception-v3-image-dc967dd42568
3. https://medium.com/@saba99/xception-cd1adc84290f